In [1]:
import json
import os
import pandas as pd
import load_geolife
import add_geohash
import create_nodes
import geohash_location_count
import geohash_user_count
import create_geohash_meet
import create_geohash_intervals
import geohash_pairs
import find_duplicates

In [9]:
def create_dirs():
    if not os.path.isdir('images/'):
        os.makedirs('images/')
    if not os.path.isdir('htmls/'):
        os.makedirs('htmls/')
    if not os.path.isdir('data/'):
        os.makedirs('data/')
    if not os.path.isdir('data/duplicates'):
        os.makedirs('data/duplicates')
    if not os.path.isdir('data/geolife_geohash_intervals'):
        os.makedirs('data/geolife_geohash_intervals')
    if not os.path.isdir('data/geolife_pairs'):
        os.makedirs('data/geolife_pairs')


def removeDuplicates(in_file, duplicates_file, out_file):
    duplicates = pd.read_csv(duplicates_file)
    df = pd.read_csv(in_file)
    df['px'] = df['Person ID'].astype('string') + "_" + df['Trajectory'].astype('string') + ".plt"
    duplicates['px'] = duplicates['user'].astype('string')+"_"+duplicates['file_name'].astype('string')
    noduplicates = df[~df['px'].isin(duplicates['px'])].copy(deep=True)
    noduplicates.to_csv(out_file,index=False)

In [3]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)

In [10]:
create_dirs()
# load_geolife.load_geolife('./geolife/Data/',"./data/geolife_gps_data.csv", "./data/geolife_labels_data.csv")
# add_geohash.add_geohash('./data/geolife_gps_data.csv', './data/geolife_geohash_size_8.csv')
# find_duplicates.find_duplicates_by_filename('./geolife/Data/', './data/duplicates/')
# find_duplicates.find_duplicates_by_content('./geolife/Data/', './data/duplicates/')
# removeDuplicates('./data/geolife_geohash_size_8.csv', './data/duplicates/geolife_user_track_duplicates_by_content.csv', './data/geolife_geohash_size_8_no_duplicates.csv')
# create_nodes.create_nodes('./data/geolife_geohash_size_8_no_duplicates.csv', './data/geolife_nodes.csv')
# geohash_location_count.geohash_location_count('./data/geolife_geohash_size_8_no_duplicates.csv', './data/geolife_geohash_8_location_counts.csv')
# geohash_user_count.geohash_user_count('./data/geolife_geohash_size_8_no_duplicates.csv', './data/geolife_geohash_8_user_counts.csv')
# count_files = create_geohash_meet.createGeohashMeetJsons('./data/geolife_geohash_size_8_no_duplicates.csv', './data/geolife_geohash_meet/')
# create_geohash_meet.createGeohashMeetCSV(count_files,'./data/geolife_geohash_meet/', './data/geolife_geohash_meet_size_8.csv')
# create_geohash_intervals.create_geohash_intervals('./data/geolife_geohash_meet_size_8.csv', './data/geolife_geohash_meet_intervals_size_8.csv')
# geohash_pairs.transform_times('./data/geolife_geohash_meet_intervals_size_8.csv','./data/geolife_geohash_intervals/','./data/geolife_geohash_8_intervals_table.csv')
# geohash_pairs.split_to_single_day('./data/geolife_geohash_8_intervals_table.csv','./data/geolife_geohash_8_intervals_single_day.csv')
# geohash_pairs.create_date_pairs('./data/geolife_geohash_8_intervals_single_day.csv','./data/geolife_geohash_8_day_meets.csv')
geohash_pairs.create_pairs('./data/geolife_geohash_8_intervals_single_day.csv','./data/geolife_pairs/', './data/geolife_geohash_8_pairs.csv')
geohash_pairs.create_edges('./data/geolife_geohash_8_pairs.csv', './data/geolife_geohash_size_8_edges.csv')

100%|██████████| 5046480/5046480 [1:08:02<00:00, 1236.26it/s]
